In [1]:
import numpy as np
import scipy
import time
import math
from scipy.stats import norm

import mystic
from mystic.solvers import DifferentialEvolutionSolver, diffev2
from mystic.strategy import Best1Bin
from mystic.monitors import Monitor,VerboseMonitor

from copy import deepcopy

from tomography import *
from constants import *
from densitymatrix import DensityMatrix
from optimization import Optimizer, function_process, function_unitary, ProcessResults, UnitaryResults
from processmatrix import complex_to_real, real_to_complex, conversion, P_matrix, X_matrix, ProcessMatrix

from nestedforloop import get_iterator
from pathlib import Path
from scipy.linalg import sqrtm

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import os
import glob

import pandas as pd

from scipy.optimize import least_squares

import fnmatch

In [2]:
# Order and names in the pseudocode
# x=(22.5,0) y=(0,-45) z=(0,0) a=(22.5,45) b=(22.5,0) c=(-22.5,-45) d=(-22.5,0) e=(45,0) f=(0,45)
# Channels: 1 2 3 4 12 14 23 24

## Matching the datafile name with the respective input and measurement basis
IBasisList=['z','e','a','y','c','f'] #in order: H=z V=-z D=x L=y A=-x R=-y
MBasisList=['x','y','z'] #in order: D L H

anglesInput=['(00.00, 00.00)', '(45.00, 00.00)', '(22.50, 45.00)','(00.00, -45.0)', '(-22.5, -45.0)', '(00.00, 45.00)']
anglesOutput=['(22.50, 00.00)','(00.00, -45.0)', '(00.00, 00.00)']
BasesI=['H', 'V', 'D', 'L', 'A', 'R']
BasesO=['D', 'L', 'H']

In [3]:
working_dir=r"C:\Users\LauraMartins\Documents\PhD\Lab\Code\Tomographies"

rhoIn=[]
rhoOUT=[]
Lambdas=[]
Rs=[]
pass_prob=[]

input_number=6
mbasis_number=6

oput=np.zeros((mbasis_number,1,2), dtype=complex)
iput=np.zeros((input_number,1,2), dtype=complex)
# The order of state: D L H A R V (To match the order we use below that is already too deep in the code)
oput[2]=H=np.array([1,0]) #z
oput[5]=V=np.array([0,1]) #e
oput[0]=D=np.array([1,1])/np.sqrt(2) #a
oput[3]=A=np.array([1,-1])/np.sqrt(2) #c
oput[1]=L=np.array([1,1j])/np.sqrt(2) #y
oput[4]=R=np.array([1,-1j])/np.sqrt(2) #f

numberofchannels=8
counts=np.zeros((numberofchannels,input_number,2,3), dtype=int)
counts_aux=np.zeros((input_number,2,3), dtype=float)
heralding_single=np.zeros((input_number,3), dtype=int)
sigma_counts=np.zeros((input_number,2,3), dtype=float)
xp_counts=np.zeros((input_number,3,2), dtype=int)
dirinv=np.zeros((input_number,2,2), dtype=complex)
efficiencies=np.zeros(8, dtype=float)

xp_counts_corrected_with_eff=[]
state=[]
tomographies=[]

qubit_number=1
repetitions=2e7 ### Not sure how to determine this

def FindingFile(containing, filenames):
    for file in filenames:
         if fnmatch.fnmatch(file, 'Bigiteration_0_'+containing+'_*'):
            return file
    print('No file containing: Bigiteration_0_', containing, '...')
    pass

### Loading data ###
#for j in range(input_number):

### State tomography ### 
for j in range(input_number):
    ### Order of the input states in the analysis: 0=H 1=V 2=D 3=L 4=A 5=R
    if j==0:
        print("Input state 0: |HxH|")
        iput[j]=H
        rhoIn.append(np.outer(H,np.conjugate(H))) 
    if j==1:
        print("Input state 1: |VxV|") 
        iput[j]=V     
        rhoIn.append(np.outer(V,np.conjugate(V))) 
    if j==2:
        print("Input state 2: |DxD|")  
        iput[j]=D
        rhoIn.append(np.outer(D,np.conjugate(D)))     
    if j==3:
        print("Input state 3: |LxL|")
        iput[j]=L
        rhoIn.append(np.outer(L,np.conjugate(L)))
    if j==4:
        print("Input state 4: |AxA|")
        iput[j]=A
        rhoIn.append(np.outer(A,np.conjugate(A)))
    if j==5:
        print("Input state 5: |RxR|")
        iput[j]=R
        rhoIn.append(np.outer(R,np.conjugate(R)))
        
    ### data is saved as "Bigiteration_0_xy" with x(y) being the measurement(input) basis
    #MBasisList=['x','y','z'] #in order: D L H
    #IBasisList=['z','e','a','y','c','f'] #in order: H=z V=-z D=x L=y A=-x R=-y
    bases=np.array([
        [MBasisList[0]+IBasisList[j],MBasisList[1]+IBasisList[j],MBasisList[2]+IBasisList[j]]])#, ## order: D, L, H

    os.chdir(working_dir+'\ProcessTomoData')
    filenames = [i for i in glob.glob("Bigiteration_0_*")]

    #for v in range(2):
    for w in range(3):
        file=FindingFile(bases[0][w], filenames)
        with open(file) as file: 
            for line in file:
                fields = line.split()
                for iter in range (len(fields)):
                    # In counts[a][b][c][d], 'a' corresponds to the channel (in order): 1 2 3 4 13 14 23 24 (defined in pseudo)
                    counts[iter][j][0][w]=fields[iter]
                    if w>0: #and v<1:
                        efficiencies[iter]+=float(fields[iter])
        counts_aux[j][0][w]=counts[-2][j][0][w] ##Channel 3 is the one recording the counts for the positive eigenvalue
        counts_aux[j][1][w]=counts[-1][j][0][w] ##Channel 4 is the one recording the counts for the negative eigenvalue
        heralding_single[j][w]=counts[1][j][0][w]    

### Normalizing the counts with the detectors efficiencies ###
efficiencies=efficiencies/np.max(efficiencies)
aux=0
for j in range(input_number):
    for w in range(3):
        heralding_single[j][w]=heralding_single[j][w]/efficiencies[1]
        counts_aux[j][0][w]=1e6*counts_aux[j][0][w]/float(efficiencies[-2]*heralding_single[j][w])
        counts_aux[j][1][w]=1e6*counts_aux[j][1][w]/float(efficiencies[-1]*heralding_single[j][w])

        if (aux<(counts_aux[j][0][w]+counts_aux[j][1][w])):#/heralding_single[j][w]):
            aux=(counts_aux[j][0][w]+counts_aux[j][1][w])#/heralding_single[j][w]
        
for j in range(input_number):
    for w in range(3):
        print('Input: ', BasesI[j], anglesInput[j], 'Output: ', BasesO[w], anglesOutput[w], 'Sum of counts normalized: ', (counts_aux[j][0][w]+counts_aux[j][1][w])/aux)#(heralding_single[j][w]*aux))
        
for j in range(input_number):
    xp_counts[j][:][:]=np.array(np.transpose(counts_aux[j][:][:])) # get the experimental counts
    xp_counts_corrected_with_eff.append(xp_counts[j])

    Iout=np.sum(counts_aux.flatten(), dtype = np.float32)
    pass_prob.append(Iout/(3*repetitions))

    statetomo=LRETomography(int(qubit_number), xp_counts[j][:][:], working_dir)
    statetomo.run() ### Runs fast maximum likelihood estimation
    tomographies.append(statetomo)
#     state.append(DensityMatrix(statetomo.quantum_state.get_density_matrix()))
    dirinv[j][:][:]=statetomo.state.state#.quantum_state.get_density_matrix()
    print('State:',  iput[j])
    print('Fast maximum likelihood estimation: \n', dirinv[j][:][:], '\n')  
    
    if j<4:
        a = np.array([[1, 1],[1,-1]])
        b = np.array([dirinv[j][0][0], dirinv[j][1][1]])
        sol_L = np.linalg.solve(a,b)

        e = np.array([[1, -1j],[1,+1j]])
        f = np.array([dirinv[j][0][1], dirinv[j][1][0]])
        sol_I = np.linalg.solve(e,f)

        c = np.array([[1, 1],[1,-1]])
        d = np.array([rhoIn[j][0][0], rhoIn[j][1][1]])
        sol_R = np.linalg.solve(c,d)
        
        Lambdas.append([sol_L[0],sol_I[0],sol_I[1],sol_L[1]])
        Rs.append([sol_R[0],np.real(rhoIn[j][1][0]),np.imag(rhoIn[j][1][0]),sol_R[1]])

        # Considering the losses, we multiply the density matrix by the probability of measuring a photon
        #print('Lambdas and pass probability: ', Lambdas[j][:], pass_prob[j])
        Lambdas[j][:]=np.array(Lambdas[j][:])*pass_prob[j]
        
iterator = get_iterator(4,3)
B=np.zeros((4,4,4,4), dtype=complex)

print('\n \n \n')
print('ORTHOGONAL STATES OVERLAP V and H: \n', np.trace(dirinv[0][:][:]@dirinv[1][:][:]))
print('ORTHOGONAL STATES OVERLAP D and A: \n', np.trace(dirinv[2][:][:]@dirinv[4][:][:]))
print('ORTHOGONAL STATES OVERLAP R and L: \n', np.trace(dirinv[3][:][:]@dirinv[5][:][:]))
print('\n')
print('ORTHOGONAL STATES OVERLAP H and D: \n', np.trace(dirinv[0][:][:]@dirinv[2][:][:]))
print('ORTHOGONAL STATES OVERLAP H and L: \n', np.trace(dirinv[0][:][:]@dirinv[3][:][:]))
print('ORTHOGONAL STATES OVERLAP D and L: \n', np.trace(dirinv[2][:][:]@dirinv[3][:][:]))
print('ORTHOGONAL STATES OVERLAP H and R: \n', np.trace(dirinv[0][:][:]@dirinv[5][:][:]))
print('ORTHOGONAL STATES OVERLAP H and A: \n', np.trace(dirinv[0][:][:]@dirinv[4][:][:]))

for m, n, j in iterator:
    temp = np.zeros((2,2), dtype=complex)
    for i in range(4):
        temp += (Rs[j][i]*Pauli[m]@Pauli[i]@Pauli[n])

    a = np.array([[1, 1],[1,-1]])
    b = np.array([temp[0][0],temp[1][1]])
    sol_diagonal = np.linalg.solve(a,b)
    c = np.array([[1, -1j],[1,1j]])
    d = np.array([temp[0][1],temp[1][0]])
    sol_anti_diagonal = np.linalg.solve(c,d)
    B[m][n][j][0]=sol_diagonal[0]
    B[m][n][j][1]=sol_anti_diagonal[0]
    B[m][n][j][2]=sol_anti_diagonal[1]
    B[m][n][j][3]=sol_diagonal[1]

    # Verification step
    ver=B[m][n][j][0]*Pauli[0]+B[m][n][j][1]*Pauli[1]+B[m][n][j][2]*Pauli[2]+B[m][n][j][3]*Pauli[3]==temp
    if False in ver:
        print('B matrix is not verifying right condidion')

Bs_new = np.transpose(np.reshape(B,(16,16)))

Kabba=np.linalg.inv(Bs_new) 

lambdas_vect=np.reshape(Lambdas,[16,1])

Chi_vector=Kabba@lambdas_vect

Input state 0: |HxH|
Input state 1: |VxV|
Input state 2: |DxD|
Input state 3: |LxL|
Input state 4: |AxA|
Input state 5: |RxR|
Input:  H (00.00, 00.00) Output:  D (22.50, 00.00) Sum of counts normalized:  0.9885814333900802
Input:  H (00.00, 00.00) Output:  L (00.00, -45.0) Sum of counts normalized:  0.9848456730659485
Input:  H (00.00, 00.00) Output:  H (00.00, 00.00) Sum of counts normalized:  0.9883036525286982
Input:  V (45.00, 00.00) Output:  D (22.50, 00.00) Sum of counts normalized:  0.9732882161499575
Input:  V (45.00, 00.00) Output:  L (00.00, -45.0) Sum of counts normalized:  0.9872494918032443
Input:  V (45.00, 00.00) Output:  H (00.00, 00.00) Sum of counts normalized:  0.9876678816045239
Input:  D (22.50, 45.00) Output:  D (22.50, 00.00) Sum of counts normalized:  1.0
Input:  D (22.50, 45.00) Output:  L (00.00, -45.0) Sum of counts normalized:  0.9922808203396197
Input:  D (22.50, 45.00) Output:  H (00.00, 00.00) Sum of counts normalized:  0.9677990364385604
Input:  L (00.00

In [4]:
# Defining of the constraints (equalities and inequalities) we want to impose in the optimization process
from mystic.penalty import quadratic_inequality,quadratic_equality

def neg_part(Y):
    return np.sum(np.clip(Y, -np.inf, 0))

# Making sure the channel is trace-preserving or trace-decreasing
def penalty1 (X):
    return np.real(np.trace(P_matrix(X)))-2
#def penalty2 (X):
#    return -neg_part(np.real(np.linalg.eig(X_matrix(X))[0]))

# Making sure the eigenvalues o Chi are positive
def penalty2 (X):
    return -np.real(np.linalg.eig(X_matrix(X))[0])[0]
def penalty3 (X):
    return -np.real(np.linalg.eig(X_matrix(X))[0])[1]
def penalty4 (X):
    return -np.real(np.linalg.eig(X_matrix(X))[0])[2]
def penalty5 (X):
    return -np.real(np.linalg.eig(X_matrix(X))[0])[3]

#def penalty3(X):
#	return neg_part(np.imag(np.linalg.eig(P_matrix(X))[0]))

# Making sure the eigenvalues o Chi are real
def penalty6(X):
	return np.imag(np.linalg.eig(X_matrix(X))[0])[0]
def penalty7(X):
	return np.imag(np.linalg.eig(X_matrix(X))[0])[1]
def penalty8(X):
	return np.imag(np.linalg.eig(X_matrix(X))[0])[2]
def penalty9(X):
	return np.imag(np.linalg.eig(X_matrix(X))[0])[3]


@quadratic_inequality(penalty1, k=1e15)
@quadratic_inequality(penalty2, k=1e15)
@quadratic_inequality(penalty3, k=1e15)
@quadratic_inequality(penalty4, k=1e15)
@quadratic_inequality(penalty5, k=1e15)

@quadratic_equality(penalty6, k=1e15)
@quadratic_equality(penalty7, k=1e15)
@quadratic_equality(penalty8, k=1e15)
@quadratic_equality(penalty9, k=1e15)

def penalty(x):
    return 0.0

In [5]:
Chi_initial=complex_to_real(Chi_vector).flatten()
# print(Chi_initial)
# Chi_initial=Chi_vector
# For perfect data the Chi matrix should be correctly inverted and the f should output 0
print('FUNCTION: ', function_process(Chi_initial, counts_aux, input_number, mbasis_number, oput, rhoIn, repetitions))
print('PENALTY: ', penalty(Chi_initial))

FUNCTION:  904105454.4890084
PENALTY:  589362074816.9025


In [6]:
bounds=[(-1,1)]*32

opt=Optimizer(Chi_initial, function_process, results=ProcessResults)
result=opt.optimize(counts_aux, input_number, mbasis_number, oput, rhoIn, repetitions, bounds=bounds, penalty=penalty)

In [7]:
# Chi_final_test=np.array([ 5.52743074e-01-1.34046541e-09j,  7.68076238e-04+1.65605631e-01j,
#  -1.50901988e-03+4.56696021e-01j,  4.52867355e-04-3.89367559e-02j,
#  -3.48189828e+00+3.18092298e+00j,  5.06187922e-02-3.44019022e-10j,
#   1.34474759e-01+1.01157407e-03j, -1.23115712e-02-4.06505827e-04j,
#   7.12458500e+00-8.67085180e+00j, -9.26453090e+00-9.34503294e+00j,
#   3.88379791e-01+1.82900957e-13j, -2.91501442e-02-1.62630901e-03j,
#  -2.29176468e-01-4.71580873e+00j, -7.36270180e+00+9.40732306e+00j,
#  -9.70540205e+00+3.11990806e+00j,  4.21028695e-03+1.51587909e-09j])

##Need to create chi in a form of a class and use .find_closest_unitary(x0=np.array([0, 0, 0]), opt2)
### When we create the class, the argument needs to be in the form: Chi_final_n=result_y/float(np.max(np.real(np.linalg.eig(P_matrix(result_y))[0])))
### was before

Chi_final_test=result.chi_matrix_normalized
Chi_final_n=ProcessMatrix(Chi_final_test)

opt2=Optimizer(np.array([0, 0, 0]), function_unitary, results=UnitaryResults)

min2=Chi_final_n.find_closest_unitary(opt2, bounds=[(-np.pi,np.pi)]*3)

print(min2.minimum())

-0.9900531199856711


In [ ]:
players=["Arya", "Cersei"]
error_runs=30

### We make the initial guess something much closer the what we know we will find in the simulations
guess_init=ProcessMatrix(result.chi_matrix).channel_2n
opt=Optimizer(guess_init, function_process, results=ProcessResults)

Chi_final_n.calculate_fidelity_error(tomographies, players, error_runs, opt, opt2, input_number, mbasis_number, oput, rhoIn, repetitions, bounds=bounds, penalty=penalty)

In [16]:
print(Chi_final_n.fidelity_mu, Chi_final_n.fidelity_std)

-0.9845522207060292 0.006461696750909757


In [ ]:
### From here we just have double checks that the matrices have physical meaning ###

In [11]:
## Defining the conditions the final Chi matrix should follow
def Trace_cond(X):
    return np.trace(P_matrix(X))

def Hermitian(X):
    return P_matrix(X)[0][1]-np.conjugate(P_matrix(X)[1][0])

def Eigenvalue(X):
    return np.linalg.eig(P_matrix(X))[0]

X=result_y
result_y=np.append(conversion(X)[0],conversion(X)[1]).flatten()

if np.imag(np.max(Eigenvalue(result_y)))==0:
    Final_Chi_vector=real_to_complex(result_y/float(np.real(np.max(Eigenvalue(result_y)))))
else:
    print('The eigenvalue cannot be complex! The result cannot be trusted! \n', Eigenvalue(result_y))

The eigenvalue cannot be complex! The result cannot be trusted! 
 [0.1006115 -1.77445849e-19j 0.09978811-6.89915889e-19j]


In [12]:
## Printing the conditions the final Chi matrix should follow
print('CONDITIONS TO VERIFY: \n')

if np.any(np.imag(Eigenvalue(Chi_initial)))!=0:
    print('The eigenvalues of the initial guess are complex! \n', Eigenvalue(Chi_initial), '\n')
    
if np.any(np.imag(Eigenvalue(result_y)))!=0:
    print('The eigenvalues of the final Chi are complex! Cannot trust the results \n', Eigenvalue(result_y), '\n')

print('Trace of P normalized (should be <= 2): \n i: ', np.trace(P_matrix(Chi_initial_n)), '\n f: ', np.trace(P_matrix(Chi_final_n)), '\n')
print('Eigenvalues of P matrix (normalized) (should be positive and <= 1): \n i: ', np.linalg.eig(P_matrix(Chi_initial_n))[0] ,'\n f: ', np.linalg.eig(P_matrix(Chi_final_n))[0], '\n')
print('Function value (should be minimized): \n i: ', f(Chi_initial, counts_aux), '\n f: ', f(result_y, counts_aux), '\n')
print('Penalty value (should be minimized): \n i: ', penalty(Chi_initial), '\n f: ', penalty(result_y), '\n')

print('\n\n')

print('ABOUT THE NON NORMALIZED P MATRIX: \n')


print('COND 1: Trace <= 2: ', Trace_cond(Chi_initial))
print('COND 2: Hermitian = 0: ', Hermitian(Chi_initial))
print('COND 6: Eigenvalue P matrix >= 0 && <=1: ', Eigenvalue(Chi_initial))
print('P_matrix initial: \n', P_matrix(Chi_initial))

print('\n')

print('COND 1: Trace <= 2: ', Trace_cond(result_y))
print('COND 2: Hermitian = 0: ', Hermitian(result_y))
print('COND 6: Eigenvalue P >= 0 && <=1: ', Eigenvalue(result_y))
print('P_matrix final: \n', P_matrix(result_y))

CONDITIONS TO VERIFY: 

The eigenvalues of the initial guess are complex! 
 [0.60156173-2.77555756e-17j 0.60156173+1.56125113e-17j] 

The eigenvalues of the final Chi are complex! Cannot trust the results 
 [0.1006115 -1.77445849e-19j 0.09978811-6.89915889e-19j] 

Trace of P normalized (should be <= 2): 
 i:  (1.9999999999999996+0j) 
 f:  (1.9918161716452136+0j) 

Eigenvalues of P matrix (normalized) (should be positive and <= 1): 
 i:  [1.-1.38777878e-17j 1.+2.16840434e-17j] 
 f:  [1.        +1.41197115e-17j 0.99181617-1.41197115e-17j] 

Function value (should be minimized): 
 i:  904105454.4890084 
 f:  21669.003522252002 

Penalty value (should be minimized): 
 i:  589362074816.9025 
 f:  0.2541130698911956 




ABOUT THE NON NORMALIZED P MATRIX: 

COND 1: Trace <= 2:  (1.2031234666666666+0j)
COND 2: Hermitian = 0:  (3.8163916471489756e-17+0j)
COND 6: Eigenvalue P matrix >= 0 && <=1:  [0.60156173-2.77555756e-17j 0.60156173+1.56125113e-17j]
P_matrix initial: 
 [[ 6.01561733e-01+0.000

In [13]:
## Printing the input states and the transformation they go through when applied the process matrix
## And printing the fidelity of the reconstructed density matrix with the ideal one
## The ideal density matrix will depend on the generated data (check mathematica program)


def Channel(X, initialstate):
    X_real, X_imag = conversion(X)
    
    Final_state=np.zeros((2,2), dtype=complex)
    for m in range (4):
        for n in range(4):
            t=n%4+4*(m%4)
            Final_state += (X_real[t]+1j*X_imag[t])*Pauli[m]@initialstate@Pauli[n]
    return Final_state

def fidelity(ideal, real):
    return ((np.trace(sqrtm(sqrtm(real)@ideal@sqrtm(real))))**2/(np.trace(ideal)*np.trace(real)))

def Chi_ideal(Th, Tv):
    r=np.array([[((np.sqrt(Th)+np.sqrt(Tv))**2)/4,0,0,(Tv-Th)/4],
              [0, 0, 0, 0],
              [0, 0, 0, 0],
              [(Tv-Th)/4, 0, 0, ((np.sqrt(Tv)-np.sqrt(Th))**2)/4]])
    return r

def verification(X, Y):
    for initial in range(input_number):
        verification = rhoIn[initial]
        Finali_state = Channel(Y, verification)
        Finalf_state = Channel(X, verification)
        print('INITIAL STATE: \n', verification)
        print('OUTPUT STATE: \n', dirinv[initial][:][:])
        print('FINAL STATE (Chi direct inversion): \n', Finali_state)
        print('FINAL STATE (Chi optmized): \n', Finalf_state)
        
        print('TRACE: \n i:', np.trace(Finali_state), '\n f:', np.trace(Finalf_state))
        
        print('\n \n')

    #F_i=fidelity(Chi_ideal(0.2,0.8), X_matrix(Y))
    #F_f=fidelity(Chi_ideal(0.2,0.8), X_matrix(Y))
    #print('FIDELITY TO THE IDEAL PROCESS MATRIX: \n i: ', F_i, '\n f: ', F_f)

    pass

In [14]:
verification(Chi_final_n, Chi_initial_n)

INITIAL STATE: 
 [[1 0]
 [0 0]]
OUTPUT STATE: 
 [[0.5490974 +4.07366572e-18j 0.45010615+1.96996737e-01j]
 [0.45010615-1.96996737e-01j 0.4509026 -4.07366572e-18j]]
FINAL STATE (Chi direct inversion): 
 [[0.5490974 +0.j         0.45010615+0.19699674j]
 [0.45010615-0.19699674j 0.4509026 +0.j        ]]
FINAL STATE (Chi optmized): 
 [[0.5578415 +0.j         0.44686695+0.19652261j]
 [0.44686695-0.19652261j 0.44124603+0.j        ]]
TRACE: 
 i: (0.9999999999999996+0j) 
 f: (0.9990875216948039+0j)

 

INITIAL STATE: 
 [[0 0]
 [0 1]]
OUTPUT STATE: 
 [[ 0.43637926-2.18421159e-17j -0.44437424-2.03628557e-01j]
 [-0.44437424+2.03628557e-01j  0.56362074+2.18421159e-17j]]
FINAL STATE (Chi direct inversion): 
 [[ 0.43637926+0.j         -0.44437424-0.20362856j]
 [-0.44437424+0.20362856j  0.56362074+0.j        ]]
FINAL STATE (Chi optmized): 
 [[ 0.43654599+0.j         -0.44180774-0.19293548j]
 [-0.44180774+0.19293548j  0.55618266+0.j        ]]
TRACE: 
 i: (0.9999999999999998+0j) 
 f: (0.9927286499504098+